A graphical interface for using the pyVRS (Python Volume-Rendered Swimmer) model of low-Reynolds number hydrodynamics to investigate stability and movement of early-stage larval morphologies approximated by chimeras of semi-ellipsoids.

In [1]:
# Set up graphics environment
%matplotlib
from mpl_toolkits import mplot3d
from matplotlib import pyplot
from matplotlib.colors import LightSource
pyplot.ioff()
# Import modules
import numpy as np
from math import pi
import os
import pyVRS.pyVRSmorph as mrph
import pyVRS.pyVRSflow as flw
from pyVRS.meshEllipsoid import chimeraEllipsoid
# Import widget infrastructure
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Using matplotlib backend: <object object at 0x7fd389a6a960>
Import of stl_utils (or numpy-stl) failed -- stl file import will not be available.


In [ ]:
# Instantiate a Params object and a GUI to modify parameters. 
params=Params(auto_plot=True)
# Restrict to a single environment, to simplify the interface
#save_all=False,s_egg=0.1,s_0=0.75,s_1=0.25,s_2=1.,
#                 Smax=1.25,s_j=1.,N0=1.,n_env=1,
#                 t_env_cycle=1.,env_cycle_ratios=np.array([1.]),g_pars=[{'g0':1.}],
#                 m_pars=[{'m0':0.15}],alpha=-0.75,c_0 = 0.5,f_2 = 1.,C = None,
#                 nruns=1,start_time=0.,end_time=10.,dt=0.05,max_step=0.005,ns=4*32,
#                 abserr=1.e-12,relerr=1.e-10,auto_plot=False
s_egg=widgets.FloatText(value=0.1,width=10,description = r"$s_{egg}$")
s_0=widgets.FloatText(value=0.75,description = r"$s_0$")
s_1=widgets.FloatText(value=0.25,description = r"$s_1$")
s_2=widgets.FloatText(value=1.0,description = r"$s_2$")
Smax=widgets.FloatText(value=1.25,description = r"$s_{max}$")
g0=widgets.FloatText(value=1.,description = r"$g_0$")
m0=widgets.FloatText(value=0.15,description = r"$m_0$")
alpha=widgets.FloatText(value=-0.75,description = r"$\alpha$")
c_0=widgets.FloatText(value=0.5,description = r"$c_0$")
f_2=widgets.FloatText(value=1.,description = r"$f_2$")

h = '30px'
L_s_egg = widgets.Label(value='Egg size',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="97%",height=h))
L_s_0 = widgets.Label(value='Size at cloning',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="95%",height=h))
L_s_1 = widgets.Label(value='Clone size',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="95%",height=h))
L_s_2 = widgets.Label(value='Size at metamorphosis',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="97%",height=h))
L_Smax = widgets.Label(value='Plot limit',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="90%",height=h))
uiL1 = widgets.VBox([L_s_egg,L_s_0,L_s_1,L_s_2,L_Smax])

L_g0 = widgets.Label(value='Growth rate',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="97%",height=h))
L_m0 = widgets.Label(value='Mortality rate',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="95%",height=h))
L_c_0 = widgets.Label(value='Cloning probability',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="95%",height=h))
L_f_2 = widgets.Label(value='Metamorphosis probability',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="97%",height=h))
uiL2 = widgets.VBox([L_g0,L_m0,L_c_0,L_f_2])


LabelObj = widgets.Label(value='Object',layout=widgets.Layout(display="flex", 
                                        justify_content="center",width="100%"))
LabelObj1 = widgets.Label(value='Object')
Space = widgets.Label(value=' ',
                         layout=widgets.Layout(display="flex", 
                                               justify_content="center", 
                                               width="100%"))
LabelModel = widgets.Label(value='Model',
                         layout=widgets.Layout(display="flex", 
                                               justify_content="flex-end", 
                                               width="150%"))
ui0 = widgets.VBox([LabelObj,LabelObj1,Space,LabelModel])
ui1 = widgets.VBox([s_egg,s_0,s_1,s_2,Smax])
ui2 = widgets.VBox([g0,m0,c_0,f_2])
ui3 = widgets.HBox([uiL1,ui1,ui2,uiL2])

def setParams(s_egg,s_0,s_1,s_2,Smax,g0,m0,alpha,c_0,f_2):
    params=Params(auto_plot=True)
    params.s_egg = s_egg
    params.s_0 = s_0
    params.s_1 = s_1
    params.s_2 = s_2
    params.Smax = Smax
    params.g_pars = [{'g0':g0}]
    params.m_pars = [{'m0':m0}]
    params.alpha = alpha
    params.c_0 = c_0
    params.f_2 = f_2
    result=ClonePDE(params)
    
out = widgets.interactive_output(setParams,{'s_egg':s_egg,'s_0':s_0,'s_1':s_1,'s_2':s_2,'Smax':Smax,
                                             'g0':g0,'m0':m0,'alpha':alpha,'c_0':c_0,'f_2':f_2})
display(ui3,out)


In [2]:
# Create a class and global instances for morphology parameters
global surf_pars, incl1_pars, incl2_pars
class CEpars():
    """
    A simple class to facilitate acquiring and passing chimera ellipsoid
    parameters with interactive_output widgets.
    """
    def __init__(self,a=50.e-6,b0=100.e-6,b1=-40.e-6,d=6e-6,nlevel0=16,nlevel1=12,
                      translate0=0.,translate1=0.,translate2=0.):
        self.a = a
        self.bs = [b0,b1]
        self.d = d
        self.nlevels = [nlevel0,nlevel1]
        self.translate = [translate0,translate1,translate2]

def set_surf_pars(a,b0,b1,d,nlevel0,nlevel1):
    global surf_pars
    surf_pars=CEpars(a=a,b0=b0,b1=b1,d=d,nlevel0=nlevel0,nlevel1=nlevel1)

def set_incl1_pars(a,b0,b1,d,nlevel0,nlevel1,translate0,translate1,translate2):
    global incl1_pars
    incl1_pars=CEpars(a=a,b0=b0,b1=b1,d=d,nlevel0=nlevel0,nlevel1=nlevel1,
                      translate0=translate0,translate1=translate1,translate2=translate2)

def set_incl2_pars(a,b0,b1,d,nlevel0,nlevel1,translate0,translate1,translate2):
    global incl2_pars
    incl2_pars=CEpars(a=a,b0=b0,b1=b1,d=d,nlevel0=nlevel0,nlevel1=nlevel1,
                      translate0=translate0,translate1=translate1,translate2=translate2)

In [3]:
# Set parameters for surface chimera
surf_a=widgets.FloatText(value=50.e-6,width=10,description = r"$a$")
surf_d=widgets.FloatText(value=6.e-6,description = r"$d$")
surf_b0=widgets.FloatText(value=100.e-6,description = r"$b_0$")
surf_nlevel0=widgets.IntText(value=16,description = r"$n_0$")
surf_b1=widgets.FloatText(value=-40.e-6,description = r"$b_1$")
surf_nlevel1=widgets.IntText(value=12,description = r"$n_1$")

ui0 = widgets.VBox([surf_a,surf_d])
ui1 = widgets.VBox([surf_b0,surf_nlevel0])
ui2 = widgets.VBox([surf_b1,surf_nlevel1])
ui012 = widgets.HBox([ui0,ui1,ui2])


out = widgets.interactive_output(set_surf_pars,{'a':surf_a,'b0':surf_b0,'b1':surf_b1,'d':surf_d,
                                                'nlevel0':surf_nlevel0,'nlevel1':surf_nlevel1})
display(ui012,out)

Output()

In [7]:
print(surf_pars.bs)
print(surf_pars.a)

[0.0001, -4e-05]
8e-05


In [8]:
surf_pars.a=0.0003

In [4]:
# Set parameters for first inclusion chimera
incl1_a=widgets.FloatText(value=30.e-6,width=10,description = r"$a$")
incl1_d=widgets.FloatText(value=5.e-6,description = r"$d$")
incl1_b0=widgets.FloatText(value=50.e-6,description = r"$b_0$")
incl1_nlevel0=widgets.IntText(value=12,description = r"$n_0$")
incl1_b1=widgets.FloatText(value=-20.e-6,description = r"$b_1$")
incl1_nlevel1=widgets.IntText(value=8,description = r"$n_1$")
incl1_translate0=widgets.FloatText(value=0.,description = r"$x$ offset")
incl1_translate1=widgets.FloatText(value=0.,description = r"$y$ offset")
incl1_translate2=widgets.FloatText(value=40.e-6,description = r"$z$ offset")

#translate=[0.,0.,40.e-6]

ui0i = widgets.VBox([incl1_a,incl1_d,incl1_translate0])
ui1i = widgets.VBox([incl1_b0,incl1_nlevel0,incl1_translate1])
ui2i = widgets.VBox([incl1_b1,incl1_nlevel1,incl1_translate2])
ui012i = widgets.HBox([ui0i,ui1i,ui2i])


outi = widgets.interactive_output(set_incl1_pars,{'a':incl1_a,'b0':incl1_b0,'b1':incl1_b1,'d':incl1_d,
                                                'nlevel0':incl1_nlevel0,'nlevel1':incl1_nlevel1,
                                                'translate0':incl1_translate0,
                                                'translate1':incl1_translate1,
                                                'translate2':incl1_translate2})
display(ui012i,outi)

Output()

In [17]:
print(incl1_pars.bs)
print(incl1_pars.a)
print(incl1_pars.translate)

[0.0001, -4e-05]
5e-05
[0.0, 0.0, 3e-05]


In [5]:
CEsurf = chimeraEllipsoid(a=surf_pars.a,bs=surf_pars.bs,d=surf_pars.d,nlevels=surf_pars.nlevels)
CEincl = chimeraEllipsoid(a=incl1_pars.a,bs=incl1_pars.bs,d=incl1_pars.d,nlevels=incl1_pars.nlevels,translate=incl1_pars.translate)

chimeraEllipsoid initialization complete...
chimeraEllipsoid initialization complete...


In [6]:
print(CEincl.vectors)

[[[ 5.00000000e-05  0.00000000e+00  4.00000000e-05]
  [ 4.96856105e-05  5.59822381e-06  4.00000000e-05]
  [ 4.97260948e-05  0.00000000e+00  5.04528463e-05]]

 [[ 4.96856105e-05  5.59822381e-06  4.00000000e-05]
  [ 4.87463956e-05  1.11260467e-05  4.00000000e-05]
  [ 4.94134275e-05  5.56755615e-06  5.04528463e-05]]

 [[ 4.87463956e-05  1.11260467e-05  4.00000000e-05]
  [ 4.71941665e-05  1.65139531e-05  4.00000000e-05]
  [ 4.84793578e-05  1.10650970e-05  5.04528463e-05]]

 ...

 [[-5.39071909e-06 -1.30143471e-05  1.62028106e-06]
  [-9.96075007e-06 -9.96075007e-06  1.62028106e-06]
  [-5.03158936e-06 -5.03158936e-06  4.07142325e-07]]

 [[ 0.00000000e+00 -7.11574191e-06  4.07142325e-07]
  [-5.39071909e-06 -1.30143471e-05  1.62028106e-06]
  [-5.03158936e-06 -5.03158936e-06  4.07142325e-07]]

 [[ 0.00000000e+00 -7.11574191e-06  4.07142325e-07]
  [-5.03158936e-06 -5.03158936e-06  4.07142325e-07]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]]


In [6]:
M = mrph.Morphology()
M.check_normals = False
M.gen_surface(vectors=CEsurf.vectors)
M.gen_inclusion(vectors=CEincl.vectors,material='lipid',immersed_in=1)

Created Medium object with parameters:
AttrDict({'density': 1030.0, 'material': 'seawater', 'immersed_in': False, 'contains': [], 'scale_factor': 1, 'offset': array([0, 0, 0]), 'rotate': array([0, 0, 0, 0]), 'layer_type': 'medium', 'transformations': [], 'stlfile': None, 'nu': 1.17e-06, 'mu': 0.0012051})
Created Surface object with parameters:
AttrDict({'density': 1070.0, 'material': 'tissue', 'immersed_in': 0, 'contains': [], 'scale_factor': 1, 'offset': array([0, 0, 0]), 'rotate': array([0, 0, 0, 0]), 'layer_type': 'surface', 'transformations': [], 'stlfile': None, 'total_area': 4.036756213581691e-08, 'total_volume': 7.282630427561315e-13, 'volume_center': array([ 2.90713553e-21, -7.19504260e-22,  2.24745998e-05]), 'tetra_project': 0.03, 'tetra_project_min': 1e-08})
Getting control and singularity points...
Added surface to layers list:
Layer -1 of type <class 'pyVRS.pyVRSmorph.Surface'>
AttrDict({'density': 1070.0, 'material': 'tissue', 'immersed_in': 0, 'contains': [], 'scale_facto

In [7]:
figure = pyplot.figure()
axes = figure.add_subplot(projection='3d')
M.plot_layers(axes=axes)

In [8]:
M.body_calcs()
M.flow_calcs(surface_layer=1)

Layer 0 of type <class 'pyVRS.pyVRSmorph.Medium'>
Layer 1 of type <class 'pyVRS.pyVRSmorph.Surface'>
F_buoyancy =  7.3585882629207806e-09
C_buoyancy =  [ 2.90713553e-21 -7.19504260e-22  2.24745998e-05]
List of all inclusions is:  [2]
F_gravity =  -7.427370197950762e-09
C_gravity =  [ 3.00426063e-21 -7.55822269e-22  2.16345790e-05]
Layer 2 of type <class 'pyVRS.pyVRSmorph.Inclusion'>
Assembling influence matrix
Calculating inverse...
Done calculating inverse.


In [81]:
# Create a class and global instances for simulation parameters
global sim_pars
class SimPars():
    """
    A simple class to facilitate acquiring and passing VRS simulation
    parameters with interactive_output widgets.
    """
    def __init__(self,dudz=0.,dvdz=0.,dwdx=0.,U0=0.,U1=0.,U2=0.,
                 Tmax=20.,cil_speed=0.5*1000*1e-6,
                 phi=pi/3.,theta=-pi/4.,psi=pi):
        self.dudz = dudz
        self.dvdz = dvdz
        self.dwdx = dwdx
        self.U0 = U0
        self.U1 = U1
        self.U2 = U2
        self.Tmax = Tmax
        self.cil_speed = cil_speed
        self.S_fixed = np.asarray([0.,0.,dudz,0.,0.,dvdz,dwdx,0.,0.])
        self.U_const_fixed = np.asarray([U0,U1,U2])
        self.XEinit = np.asarray([0.,0.,0.,phi,theta,psi])

def set_sim_pars(dudz,dvdz,dwdx,Tmax,cil_speed,phi,theta,psi):
    global sim_pars
    sim_pars=SimPars(dudz=dudz,dvdz=dvdz,dwdx=dwdx,
                 Tmax=Tmax,cil_speed=cil_speed,
                 phi=phi,theta=theta,psi=psi)

print('Simulation parameters class defined...')

Simulation parameters class defined...


In [82]:
# Set parameters for first inclusion chimera
spars_dudz=widgets.FloatText(value=0.,width=10,description = r"$\frac{dU}{dz}$")
spars_dvdz=widgets.FloatText(value=0.,description = r"$\frac{dV}{dz}$")
spars_dwdx=widgets.FloatText(value=0.,description = r"$\frac{dW}{dx}$")
spars_Tmax=widgets.FloatText(value=10,description = r"$T_{max}$")
spars_cil_speed=widgets.FloatText(value=500.e-6,description = r"$V_{cilia}$")
spars_phi=widgets.FloatText(value=pi/3.,description = r"$phi$")
spars_theta=widgets.FloatText(value=pi/4.,description = r"$\theta$")
spars_psi=widgets.FloatText(value=pi,description = r"$\psi$")

#translate=[0.,0.,40.e-6]

ui0s = widgets.VBox([spars_dudz,spars_phi,spars_Tmax])
ui1s = widgets.VBox([spars_dvdz,spars_theta,spars_cil_speed])
ui2s = widgets.VBox([spars_dwdx,spars_psi])
ui012s = widgets.HBox([ui0s,ui1s,ui2s])


outs = widgets.interactive_output(set_sim_pars,{'dudz':spars_dudz,'dvdz':spars_dvdz,'dwdx':spars_dwdx,'Tmax':spars_Tmax,
                                                'cil_speed':spars_cil_speed,'phi':spars_phi,
                                                'theta':spars_theta,
                                                'psi':spars_psi})
display(ui012s,outs)

Output()

In [109]:
global Sim
button = widgets.Button(description="Reset")
output = widgets.Output()
button2 = widgets.Button(description="Run")
output2 = widgets.Output()

buttons = widgets.HBox([button, output, button2, output2])
display(buttons)

@output.capture()
def on_button_clicked(b):
    global Sim, sim_pars
    Sim = flw.VRSsim(morph=M)

@output2.capture()
def on_button_clicked2(b):
    global Sim,sim_pars
    Sim.run(XEinit=sim_pars.XEinit,Tmax=sim_pars.Tmax,cil_speed=sim_pars.cil_speed,U_const_fixed=sim_pars.U_const_fixed,S_fixed=sim_pars.S_fixed)

button.on_click(on_button_clicked)
button2.on_click(on_button_clicked2)

In [110]:
from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser('/home/dg/Desktop/test/subtest')
display(fc)

# Print the selected path, filename, or both
#print(fc.selected_path)
#print(fc.selected_filename)
#print(fc.selected)

# Change defaults and reset the dialog
#fc.default_path = '/home/dg/Desktop/test'
fc.default_filename = 'ESfig.svg'
fc.reset()

# Shorthand reset
#fc.reset(path='/Users/crahan/', filename='output.txt')

# Restrict navigation to /Users
#fc.sandbox_path = '/home/dg/Desktop/test'

# Change hidden files
fc.show_hidden = False #True

# Customize dir icon
#fc.dir_icon = '/home/dg/Desktop/test'
fc.dir_icon = '/'
fc.dir_icon_append = True

# Switch to folder-only mode
#fc.show_only_dirs = True

# Set multiple file filter patterns (uses https://docs.python.org/3/library/fnmatch.html)
fc.filter_pattern = ['*.jpg', '*.png','*.svg']

# Change the title (use '' to hide)
fc.title = '<b>Save Figure</b>'

# Sample callback function
def change_title(chooser):
    #chooser.title = '<b>Callback function executed</b>'
    #print(dir(chooser))
    Sim.fig.savefig(chooser.selected,format='svg',dpi=800)
    

# Register callback function
fc.register_callback(change_title)


FileChooser(path='/home/dg/Desktop/test/subtest', filename='', title='', show_hidden=False, select_desc='Selec…